In [ ]:
import os
os.chdir("../")

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset
from tslearn.clustering import TimeSeriesKMeans
import matplotlib.pyplot as plt

In [ ]:
class UCI_HARDataset(Dataset):

    def __init__(self,
            dataset_dir: str = None,
            split: str = "train",
            wsize: int = 10,
            wstride: int = 1
            ) -> None:
        super().__init__()

        '''
            UCI-HAR dataset handler

            Inputs:
                dataset_dir: Directory of the prepare_har_dataset.py
                    processed dataset.
                wsize: window size
                wstride: window stride
        '''

        self.wsize = wsize
        self.wstride = wstride

        # load dataset
        files = filter(
            lambda x: "sensor.npy" in x,
            os.listdir(os.path.join(dataset_dir, "UCI HAR Dataset", split)))
        
        splits = [0]

        STS = []
        SCS = []
        for f in files:
            sensor_data = np.load(os.path.join(dataset_dir, "UCI HAR Dataset", split, f))
            STS.append(sensor_data)
            SCS.append(np.load(os.path.join(dataset_dir, "UCI HAR Dataset", split, f.replace("sensor", "class"))))

            splits.append(splits[-1] + sensor_data.shape[0])

        self.splits = np.array(splits)

        self.STS = np.concatenate(STS)
        self.SCS = np.concatenate(SCS)

        self.indices = np.arange(self.SCS.shape[0])
        for i in range(wsize * wstride):
            self.indices[self.splits[:-1] + i] = 0
        self.indices = self.indices[np.nonzero(self.indices)]

    def __len__(self):
        return self.indices.shape[0]
    
    def __getitem__(self, index: int) -> tuple[np.ndarray, np.ndarray]:

        first = self.indices[index]-self.wsize*self.wstride
        last = self.indices[index]

        return self.STS[first:last:self.wstride,:], self.SCS[first:last:self.wstride]
    
    def ts_from_array(self, indexes: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
        assert len(indexes.shape) == 1 # only accept 1-dimensional arrays

        return_sts = np.empty((indexes.shape[0], self.wsize, self.STS.shape[-1]))
        return_scs = np.empty((indexes.shape[0], self.wsize))

        for i, id in enumerate(indexes):
            ts, c = self[id]
            return_scs[i] = c
            return_sts[i] = ts

        return return_sts, return_scs
    
    def getWindowsIndex(self):

        id = []
        cl = []
        for i, ix in enumerate(self.indices):
            if np.unique(self.SCS[(ix-self.wsize*self.wstride):ix]).shape[0] == 1:
                id.append(i)
                cl.append(self.SCS[ix])
        
        return np.array(id), np.array(cl)

In [ ]:
ds = UCI_HARDataset("./datasets/UCI-HAR/", split="train", wsize=64)

In [ ]:
wid, wcl = ds.getWindowsIndex()

In [ ]:
def obtain_barycenter(dataset, window_id, window_lb, n_random=100):
    
    np.random.seed(0)

    selected = np.empty((np.unique(window_lb).shape[0], dataset.wsize, dataset.STS.shape[-1]))

    for i, c in enumerate(np.unique(window_lb)):
        # get the random windows for the class c

        rw = np.random.choice(window_id[window_lb == c].reshape(-1), n_random)

        ts, cs = dataset.ts_from_array(rw)

        km = TimeSeriesKMeans(n_clusters=1, verbose=True, random_state=1, metric="dtw", n_jobs=-1)
        km.fit(ts)

        selected[i] = km.cluster_centers_[0]
    
    return selected

In [ ]:
barycenters = obtain_barycenter(ds, wid, wcl, 1000)

In [ ]:
barycenters.shape

In [ ]:
fig, ax = plt.subplots(nrows=1)
[ax.plot(barycenters[0,:,d]) for d in range(6)]

In [ ]:
from s3ts.api.encodings import compute_DM

In [ ]:
class UCI_HARDataset_DM(UCI_HARDataset):

    def __init__(self,
            dataset_dir: str = None,
            split: str = "train",
            wsize: int = 10,
            wstride: int = 1,
            patterns: np.ndarray = None,
            w: float = 0.2
            ) -> None:
        super().__init__(dataset_dir=dataset_dir, split=split, wsize=wsize, wstride=wstride)

        '''
            UCI-HAR dataset handler, with DF computation

            Inputs:
                dataset_dir: Directory of the prepare_har_dataset.py
                    processed dataset.
                wsize: window size
                wstride: window stride
                patterns: patterns used for DF computation
                w: online dtw forgetting parameter
        '''

        assert patterns.shape[1] == wsize

        self.rho = w

        # compute and save DM to disk, one per split
        if not os.path.exists(os.path.join(dataset_dir, "cached_df")):
            os.mkdir(os.path.join(dataset_dir, "cached_df"))

        self.cache_dir = os.path.join(dataset_dir, "cached_df")
        self.split = split
        self.patterns = patterns

        for s in range(self.splits.shape[0] - 1):
            save_path = os.path.join(dataset_dir, f"cached_df/{split}_split{s}.npy")
            self.compute_dm_cache(patterns, self.splits[s:s+2], save_path)

        self.loaded_split = None
        self.temp = None

    def compute_dm_cache(self, pattern, split, save_path):
        DM = compute_DM(self.STS[split[0]:split[1]].T, pattern.transpose(0, 2, 1), rho=self.rho)

        with open(save_path, "wb") as f:
            np.save(f, DM.transpose((0, 2, 1)))

    def __getitem__(self, index: int) -> tuple[np.ndarray, np.ndarray]:

        id = self.indices[index]

        # identify the split of the index

        s = np.argwhere(self.splits > id)[0, 0] - 1
        first = id - self.wsize*self.wstride - self.splits[s]
        last = id - self.splits[s]

        if self.loaded_split != s:
            self.temp = np.load(os.path.join(self.cache_dir, f"{self.split}_split{s}.npy"))
            self.loaded_split = s

        return self.temp[:, first:last:self.wstride, :], self.STS[first:last:self.wstride, :], self.SCS[id]

In [ ]:
ds_df = UCI_HARDataset_DM("./datasets/UCI-HAR/", split="train", wsize=64, patterns=barycenters, wstride=1)

In [ ]:
from IPython.display import display, clear_output

In [ ]:
from time import time

In [ ]:
a=time()
for i in range(1000):
    ds_df[i]
    print(f"\r{i}: {(i+1)/(time()-a)} per/s", end="", flush=True)

In [ ]:
len(ds_df)

In [ ]:
from storage.label_mappings import UCI_HAR_LABELS

In [ ]:
plot, ax = plt.subplots(nrows=2, ncols=3)

ax_images = []
for i in range(2):
    for j in range(3):
        if i==0:
            ax_images.append(ax[i, j].imshow(np.random.randn(64, 64), vmin=0, vmax=64))
        if i==1:
            ax_images.append(ax[i, j].imshow(np.random.randn(64, 64), vmin=0, vmax=32))

        ax_twin = ax[i, j].twinx()
        ax_twin.set_xlim([0, 64])
        ax_twin.set_ylim([0, 64])
        [ax_twin.plot(barycenters[i*3+j,:,h]*20 + 15, np.arange(64)) for h in range(6)]

for i in range(10000, 20000, 16):
    for col in range(3):
        df, ts, c = ds_df[i]
        for h in range(6):
            ax_images[h].set_data(df[h].T)

    display(plot)    
    clear_output(wait = True)